In [12]:
from sklearn.metrics import mean_squared_error
import pandas as pd



In [13]:
def data_read_pre_processing(file_path):
    
   
    #reading the data from the defined path
    df = pd.read_parquet(file_path)
    
    #Converting the columns to timestamp
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    
    #Calculating duration of the trip
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df['duration'] = df.duration.apply(lambda td: td.total_seconds()/60)
    
    #filetring the data 
    df = df[(df.duration>=1) & (df.duration<=60)]
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [14]:
def preprocessor(df,dv):

    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']#['PULocationID','DOLocationID']
    numerical = ['trip_distance']
    train_dicts = df[categorical+numerical].to_dict(orient = 'records')
    return dv.transform(train_dicts)

def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [15]:
df = data_read_pre_processing("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-03.parquet")

In [16]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)


run_id = '0121cbf909694d9b9aaac20ca7bc5491'


#client.list_experiments()
client.download_artifacts(run_id = run_id, path = 'preprocessor', dst_path='.')

'/workspaces/Mlops_Zoom_camp/02-Experiment Tracking/preprocessor'

In [17]:
import pickle
import mlflow
with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [18]:
X_test = preprocessor(df, dv)

In [19]:
target = "duration"
y_test = df[target].values

In [21]:
model_name = "nyc-taxi-regressor"
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

MlflowException: Registered Model with name=nyc-taxi-regressor not found

In [10]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

MlflowException: Registered Model with name=nyc-taxi-regressor not found

In [11]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True
)

/tmp/ipykernel_4024/3610659809.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(
2025/12/12 03:00:30 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/12/12 03:00:30 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<ModelVersion: aliases=[], creation_timestamp=1765508163691, current_stage='Production', deployment_job_state=None, description='', last_updated_timestamp=1765508430126, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='', run_link='', source=('/workspaces/Mlops_Zoom_camp/02-Experiment '
 'Tracking/mlruns/1/models/m-6a291090fbe448588df80a815cfa2010/artifacts'), status='READY', status_message=None, tags={}, user_id=None, version=4>